In [44]:
import nltk

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
 
# dl = nltk.downloader.Downloader('http://nltk.org/nltk_data/index.xml')
# # nltk.download('stopwords')
# dl.download('stopwords')

from nltk.corpus import stopwords
stopwords = stopwords.words('english')    
#print(stopwords) # see default stopwords

def stop_word_removal(row):
    token = [token for token in row if token not in stopwords]
    token = filter(None, token)
    return token

In [45]:
# Params for bert model and tokenization
Nsamp = 200 # number of samples to generate in each class - 'spam', 'not spam'
maxtokens = 200 # the maximum number of tokens per document
maxtokenlen = 100 # the maximum length of each token
Enron_Samp = 1000 

In [46]:
def reg_expressions(row):
    tokens = []
    try:
        for token in row:
            token = token.lower()
            token = re.sub(r'[\W\d]', "", token)
            token = token[:maxtokenlen] # truncate token
            tokens.append(token)
    except:
        token = ""
        tokens.append(token)
    return tokens

In [47]:
def tokenize(row):
    if row is None or row is '':
        tokens = ""
    else:
        try:
            tokens = row.split(" ")[:maxtokens]
        except:
            tokens=""
    return tokens

In [48]:
import re, string

def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

#jose data

In [49]:
import os
jose_phishing_files = []

for dirname, _, filenames in os.walk('../data/jose_phishing_html'):
    for filename in filenames:
        if filename.startswith('phishing'):
            jose_phishing_files.append(os.path.join(dirname, filename))
jose_phishing_files[0]    

'../data/jose_phishing_html/phishing5.mbox-1805.eml.html'

In [50]:
def get_data_content(files):
    data = []
    for file in files:
        with open(file, 'r') as f:
            data.append(f.read())
    return data
    
jose_file_contents = get_data_content(jose_phishing_files)
len(jose_file_contents)

10147

In [51]:
import pandas as pd
jose_bodies_df = pd.DataFrame(jose_file_contents[:Nsamp],columns=["message"],dtype=str)
# jose_bodies_df = pd.DataFrame(jose_bodies_df[1:])

jose_bodies_df.head() # yo

,message
0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Transitional//EN"">\n<HTML><HEAD>\n<META http-equiv=Content-Type content=""text/html; charset=windows-1250"">\n<STYLE></STYLE>\n</HEAD>\n<BODY>\n<font color=""white"">More fool you </font><font color=""white"">More honoured in the breach than in the observanc..."
1,"<html dir=""ltr"">\n<head>\n<meta http-equiv=""Content-Type"" content=""text/html; charset=iso-8859-1"">\n<style id=""owaParaStyle"">P {\n\tMARGIN-BOTTOM: 0px; MARGIN-TOP: 0px\n}\nP {\n\tMARGIN-BOTTOM: 0px; MARGIN-TOP: 0px\n}\n</style>\n</head>\n<body fPStyle=""1"" ocsi=""0"">\n<div style=""direction: ltr;fo..."
2,"<TABLE cellSpacing=0 cellPadding=0 width=""100%"" border=0>\n<TBODY>\n<TR>\n<TD>\n<TABLE cellSpacing=0 cellPadding=0 width=""100%"" border=0>\n<TBODY>\n<TR>\n<TD>\n<TABLE cellSpacing=0 cellPadding=2 width=""100%"" border=0>\n<TBODY>\n<TR>\n<TD><FONT face=""Verdana, sans-serif"" color=#666666 size=1><B>e..."
3,"Your mailbox quota has been exceeded the storage limit which is 20GB as set\n\n\n\nby your administrator, You are currently running on 20.9GB.\n\n\n\nYou may not be able to send or receive new mails until you re-validate your\n\nmailbox.\n\n\n\nTo re-activate your account please click the link b..."
4,"<style type=""text/css"">\n.n {\n\tcolor: #FF0000;\n}\n</style>\n<tbody id=""yiv9779011297yui_3_16_0_1_1441348365671_2185""><tr id=""yiv9779011297yui_3_16_0_1_144134865671_2338""><td id=""yiv9779011297yui_3_16_0_1_1441348365671_2337""> </td></tr><tr id=""yiv9779011297yui_3_16_0_1_1441348365671_2220""><td ..."


In [52]:
len(jose_bodies_df)

200

#Enron email data.

In [53]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
filepath = "../data/enron/emails.csv"

# Read the enron data into a pandas.DataFrame called emails
emails = pd.read_csv(filepath)

print("Successfully loaded {} rows and {} columns!".format(emails.shape[0], emails.shape[1]))
emails.head()

Successfully loaded 517401 rows and 2 columns!


,file,message
0,allen-p/_sent_mail/1.,"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-T..."
1,allen-p/_sent_mail/10.,"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: john.lavorato@enron.com\nSubject: Re:\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen..."
2,allen-p/_sent_mail/100.,"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>\nDate: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: leah.arsdall@enron.com\nSubject: Re: test\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K ..."
3,allen-p/_sent_mail/1000.,"Message-ID: <13505866.1075863688222.JavaMail.evans@thyme>\nDate: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: randall.gay@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-..."
4,allen-p/_sent_mail/1001.,"Message-ID: <30922949.1075863688243.JavaMail.evans@thyme>\nDate: Thu, 31 Aug 2000 05:07:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: greg.piper@enron.com\nSubject: Re: Hello\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K A..."


In [54]:
#emails.loc[100]["message"]


In [55]:
import email

def extract_messages(df):
    messages = []
    for item in df["message"]:
        # Return a message object structure from a string
        e = email.message_from_string(item)    
        # get message body  
        message_body = e.get_payload()
        messages.append(message_body)
    print("Successfully retrieved message body from e-mails!")
    return messages

bodies = extract_messages(emails)

Successfully retrieved message body from e-mails!


In [56]:
len(bodies)

517401

In [57]:
import random
bodies_df = pd.DataFrame(random.sample(bodies, Enron_Samp))

# expand default pandas display options to make emails more clearly visible when printed
pd.set_option('display.max_colwidth', 300)

bodies_df.head() # 

,0
0,"No, you can keep the diskettes. I thought we amended the Cap. Alloc. in\nregards to some bank work we did; however, it might've been the Gas Gathering\nAgrmt. Will check my files and let you know what I find later today.\n---------- Original Text ----------\n\nFrom: ""Gerald Nemec"" <Gerald.Neme..."
1,"Adrian has incorporated my edits, so I send it to you for any FINAL changes \nyou recommend. Please contact Adrian directly--I am leaving town and he is \nanxious to complete this and put it in the public domain.\n\nI'd look carefully at the last 2 1/2 pages on alternatives to price \ncaps--sta..."
2,"---------------------- Forwarded by Eric Bass/HOU/ECT on 03/30/2000 01:38 PM \n---------------------------\n \n\tEnron North America Corp.\n\t\n\tFrom: Timothy Blanchard @ EES 03/28/2000 01:28 PM\n\t\n\nTo: val.generes@ac.com, Matthew Lenhart/HOU/ECT@ECT, Eric Bass/H..."
3,"otherwise known as CA Development I and CA Development II\n---------------------- Forwarded by Kay Mann/Corp/Enron on 12/21/2000 12:13 \nPM ---------------------------\n\n\n""George Kutzschbach"" <gkutzschbach@fulbright.com> on 12/21/2000 12:00:13 PM\nTo: Kay.Mann@enron.com\ncc: DPedigo@INTERGEN.c..."
4,"FYI.\n \nThanks,\n \n\nBert Stipelcovich\n\nManager, Information Systems\n\nBridgeline Holdings, L.P.\n\n832.851.2880 Office 713.516.2230 Cell \n\n \n\n-----Original Message-----\nFrom: ICE User Administration [mailto:ICE-Administration@intcx.com]\nSent: Monday, April 22, 2002 11:11 AM\nTo: ..."


#fraudulent corpus

In [58]:
filepath = "../data/fradulent_emails.txt"
with open(filepath, 'r',encoding="latin1") as file:
    data = file.read()
    
# split on a code word appearing close to the beginning of each email
fraud_emails = data.split("From r")
len(fraud_emails)

3978

In [59]:
#fraud_emails[1]


In [60]:
fraud_bodies = extract_messages(pd.DataFrame(fraud_emails[:Nsamp + 1],columns=["message"],dtype=str))
fraud_bodies_df = pd.DataFrame(fraud_bodies[1:])

fraud_bodies_df.head() # yo

Successfully retrieved message body from e-mails!


,0
0,"FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-27-587908.\nE-MAIL: (james_ngola2002@maktoob.com).\n\nURGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\n\nDEAR FRIEND,\n\nI AM ( DR.) JAMES NGOLA, THE PERSONAL ASSISTANCE TO THE LATE CONGOLESE (PRESIDENT LAURENT KABILA) WHO WAS ASSASSINATED BY HIS BODY G..."
1,"Dear Friend,\n\nI am Mr. Ben Suleman a custom officer and work as Assistant controller of the Customs and Excise department Of the Federal Ministry of Internal Affairs stationed at the Murtala Mohammed International Airport, Ikeja, Lagos-Nigeria.\n\nAfter the sudden death of the former Head of s..."
2,"FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ELEME KINGDOM \nCHIEF DANIEL ELEME, PHD, EZE 1 OF ELEME.E-MAIL \nADDRESS:obong_715@epatra.com \n\nATTENTION:PRESIDENT,CEO Sir/ Madam. \n\nThis letter might surprise you because we have met\nneither in person nor by correspondence. But I believe\nit is..."
3,"FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ELEME KINGDOM \nCHIEF DANIEL ELEME, PHD, EZE 1 OF ELEME.E-MAIL \nADDRESS:obong_715@epatra.com \n\nATTENTION:PRESIDENT,CEO Sir/ Madam. \n\nThis letter might surprise you because we have met\nneither in person nor by correspondence. But I believe\nit is..."
4,"Dear sir, \n \nIt is with a heart full of hope that I write to seek your help in respect of the context below. I am Mrs. Maryam Abacha the former first lady of the former Military Head of State of Nigeria General Sani Abacha whose sudden death occurred on 8th of June 1998 as a result of cardiac ..."


#IWSPA


In [61]:
# paths of the input files
no_head_train_path_0 = '../data/phishing/IWSPA-AP-traindata/phish/'
no_head_train_path_1 = '../data/phishing/IWSPA-AP-traindata/legit/'
head_train_path_0 = '../data/phishing/Dataset_Full_Header_Training/Dataset_Submit_Phish/'
head_train_path_1 = '../data/phishing/Dataset_Full_Header_Training/Dataset_Submit_Legit/'
no_head_test_path = '../data/phishing/IWSPA-APTestData/testdata_noheaders/'
head_test_path = '../data/phishing/IWSPA-APTestData/testdata_fullheaders/'

In [62]:
import os, re, string
import numpy as np

In [63]:
def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

In [64]:

def get_data(path):
    text_list = list()
    files = os.listdir(path)
    for text_file in files:
        file_path = os.path.join(path, text_file)
        read_file = open(file_path,'r+')
        read_text = read_file.read()
        read_file.close()
        cleaned_text = clean_text(read_text)
        text_list.append(cleaned_text)
    return text_list, files

In [65]:
no_head_train_phish, temp = get_data(no_head_train_path_0)
no_head_train_legit, temp = get_data(no_head_train_path_1)
#head_train_0, temp = get_data(head_train_path_0)
#head_train_1, temp = get_data(head_train_path_1)
no_head_test, no_head_files = get_data(no_head_test_path)
#head_test, head_files = get_data(head_test_path)

In [66]:
# no_head_train = no_head_train_phish + no_head_train_legit
# no_head_labels_train = ([0] * len(no_head_train_phish)) + ([1] * len(no_head_train_legit))

#head_train = head_train_0 + head_train_1
#head_labels_train = ([0] * len(head_train_0)) + ([1] * len(head_train_1))

In [67]:
import pandas as pd
aw_legit_df = pd.DataFrame(no_head_train_legit[:Nsamp],columns=["message"],dtype=str)
aw_phish_df = pd.DataFrame(no_head_train_phish[:Nsamp],columns=["message"],dtype=str)
# jose_bodies_df = pd.DataFrame(jose_bodies_df[1:])

aw_legit_df.head() 

,message
0,hello everyone just wanted to see if we can reach a decision here thank you
1,gareth it seems to me from reading your email that you do not realize that this is precisely what magiccscengen already does ie it uses the scaling method that ben santer and i invented in the late 1980s to get time dependent patterns of future climate change i am attaching a description of the ...
2,daniele milan updated a event in the tentative activity calendar jun 09 2014 until jun 13 2014 knb advanced trainingyou are receiving this notification because you are watching the following tentative activity sent by atlassian confluence 544 the enterprise wiki
3,thank you 20 jordan kaplan national finance director democratic national committee 202 4885002 o 312 3390224 c userdomaincom
4,does stuart make many mistakes sammi fuck i hate her


In [68]:
aw_legit_df.head() 

,message
0,hello everyone just wanted to see if we can reach a decision here thank you
1,gareth it seems to me from reading your email that you do not realize that this is precisely what magiccscengen already does ie it uses the scaling method that ben santer and i invented in the late 1980s to get time dependent patterns of future climate change i am attaching a description of the ...
2,daniele milan updated a event in the tentative activity calendar jun 09 2014 until jun 13 2014 knb advanced trainingyou are receiving this notification because you are watching the following tentative activity sent by atlassian confluence 544 the enterprise wiki
3,thank you 20 jordan kaplan national finance director democratic national committee 202 4885002 o 312 3390224 c userdomaincom
4,does stuart make many mistakes sammi fuck i hate her


In [69]:
# legit
EnronEmails = bodies_df.iloc[:,0]#.apply(tokenize)
# EnronEmails = bodies_df.iloc[:,0].apply(tokenize)
# EnronEmails = EnronEmails.apply(stop_word_removal)
# EnronEmails = EnronEmails.apply(reg_expressions)
# EnronEmails = EnronEmails.sample(len(EnronEmails))

In [70]:
# iwspa
IwspaEmails_legit =  aw_legit_df.iloc[:, 0]#.apply(tokenize)
# IwspaEmails_legit = IwspaEmails_legit.sample(len(IwspaEmails_legit))

In [71]:
# phishing
fraudEmails = fraud_bodies_df.iloc[:,0]#.apply(tokenize)
# fraudEmails = fraud_bodies_df.iloc[:,0].apply(tokenize)
# fraudEmails = fraudEmails.apply(stop_word_removal)
# fraudEmails = fraudEmails.apply(reg_expressions)
# fraudEmails = fraudEmails.sample(len(fraudEmails))

In [72]:
# iwspa
IwspaEmails_phish =  aw_phish_df.iloc[:, 0]#.apply(tokenize)
# IwspaEmails_phish = IwspaEmails_phish.sample(len(IwspaEmails_phish))

In [73]:
# jose
JoseEmails = jose_bodies_df.iloc[:, 0]#.apply(tokenize)
# JoseEmails = JoseEmails.sample(len(JoseEmails))


raw_data = pd.concat([fraudEmails, IwspaEmails_phish, JoseEmails, EnronEmails, IwspaEmails_legit], axis=0).values


In [74]:
len(raw_data)

1800

In [75]:
# 1:spam 0:not spam
print("Shape of combined data represented as numpy array is:")
print(raw_data.shape)
#print("Data represented as numpy array is:")
#print(raw_data)

# corresponding labels
Categories = ['spam','notspam']
#header = ([1]*Nsamp)
#header.extend(([0]*Nsamp))
header = ([1]*(len(fraudEmails) + len(IwspaEmails_phish) + len(JoseEmails)))
header.extend(([0]*(len(EnronEmails) + len(IwspaEmails_legit))))

Shape of combined data represented as numpy array is:
(1800,)


In [76]:
type(raw_data[10])

str

In [77]:
# function for shuffling data in unison with labels/header
def unison_shuffle(a, b):
    p = np.random.permutation(len(b))
    data = a[p]
    header = np.asarray(b)[p]
    return data, header

# function for converting data into the right format, due to the difference in required format from sklearn models
# we expect a single string per email here, versus a list of tokens for the sklearn models previously explored
def convert_data(raw_data,header):
    converted_data, labels = [], []
    for i in range(raw_data.shape[0]):
        # out = ' '.join(str(raw_data[i]))
        out = str(raw_data[i])
        converted_data.append(out)
        labels.append(header[i])
        #print(i)
    converted_data = np.array(converted_data, dtype=object)[:, np.newaxis]
    
    return converted_data, np.array(labels)

raw_data, header = unison_shuffle(raw_data, header)

# split into independent 70% training and 30% testing sets
idx = int(0.7*raw_data.shape[0])
# 70% of data for training
train_x, train_y = convert_data(raw_data[:idx],header[:idx])
# remaining 30% for testing
test_x, test_y = convert_data(raw_data[idx:],header[idx:])

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
# print(train_x)
# print(train_y[:5])
print(train_y.shape)


train_x/train_y list details, to make sure it is of the right form:
1260
(1260,)


In [78]:
np.save('../output/train_x.npy', train_x)
np.save('../output/train_y.npy', train_y)
np.save('../output/test_x.npy', test_x)
np.save('../output/test_y.npy', test_y)

In [79]:
store = pd.concat([pd.DataFrame(train_x), pd.DataFrame(train_y)], axis=1).values
store1 = pd.concat([pd.DataFrame(test_x), pd.DataFrame(test_y)], axis=1).values

In [80]:
store[0]


array(["Tracy,\n\nPlease add Blake Walker and Keith Marlow to your distribution of future correspondence regarding the plan presentation.  Also, while I'm on maternity leave, Blake will be the EEOS contact.\n\nThanks,\n\nErin\n\n -----Original Message-----\nFrom: \tGeaccone, Tracy  \nSent:\tThursday, October 18, 2001 11:07 AM\nTo:\tMcCarty, Danny; Cordes, Bill; Stanley, Brian; FOWLER, PEGGY; Garrison, John L; Hughes, James A.; Umanoff, Adam\nCc:\tAnderson, Michael; Sommers, Jeffrey E.; Peters, Jerry; Copeland, Erin; Dodson, Keith; Piro, Jim; Lindholm, Tod A.; Armstrong, Julie; Stark, Cindy; Hayslett, Rod; Stark, Cindy\nSubject:\t2002 Plan Lay Presentation Run Through \n\nStan would like to have a run through of the presentation for the Lay reviews on November 1.  Please see the agenda attached below for your respective group's time.  We will be meeting in EB 49C2.  If you need to call in, the number is below.\nIf you can not attend, please send a representative.  \n\n\n \nCALL IN:\nPAR

In [81]:
csv = pd.DataFrame(store)
csv.to_csv('../output/train_1.csv')

In [82]:
csv = pd.DataFrame(store1)
csv.to_csv('../output/train_2.csv')

In [83]:
train_1 = pd.read_csv('../output/train_1.csv')
train_1.head()

,Unnamed: 0,0,1
0,0,"Tracy,\n\nPlease add Blake Walker and Keith Marlow to your distribution of future correspondence regarding the plan presentation. Also, while I'm on maternity leave, Blake will be the EEOS contact.\n\nThanks,\n\nErin\n\n -----Original Message-----\nFrom: \tGeaccone, Tracy \nSent:\tThursday, Oc...",0
1,1,"Guz, what's new?\n\n_____________________________________________\nSign up for your gosfgiants.com e-mail today!\nhttp://www.gosfgiants.com",0
2,2,"---------------------- Forwarded by Eric Bass/HOU/ECT on 03/30/2000 01:38 PM \n---------------------------\n \n\tEnron North America Corp.\n\t\n\tFrom: Timothy Blanchard @ EES 03/28/2000 01:28 PM\n\t\n\nTo: val.generes@ac.com, Matthew Lenhart/HOU/ECT@ECT, Eric Bass/H...",0
3,3,fyi amy k dacey chief executive officer democratic national committee 430 s capitol street se washington dc 20003 2025287492 c 2023142263 o userdomaincommailtouserdomaincom cidimage001png01cf74df0abf9350,0
4,4,"DAYO NAPS\n234 80 372 20807\n\nDear Sir,\n\nI am Mr. Dayo Naps, an official with one of the APEX banks in Nigeria.My colleagues and I have an urgent and very confidential \nbusiness proposal for you. On the 6th of June 1998 an American miner, Mr. Travis Myers ran an account with us and\nhis pres...",1


In [84]:
train_2 = pd.read_csv('../output/train_2.csv')
train_2.head()

,Unnamed: 0,0,1
0,0,"Global No. 96034155 has been changed to 01/01/2000.\n\nSylvia\n---------------------- Forwarded by Sylvia A Campos/HOU/ECT on 03/06/2000 \n10:16 AM ---------------------------\n\n\nSylvia A Campos\n03/03/2000 08:58 AM\nTo: Dan Junek/HOU/ECT@ECT, Chris Germany/HOU/ECT@ECT\ncc: Cathy Sprowls/HOU/E...",0
1,1,"<< > You have just received the ""Aggie Virus""\n >\n > As we don't have any programming experience, this virus works on the\n > honor system. Please delete all the files from your hard drive and\n > manually forward this virus to everyone on your mailing list.\n >\n > Thanks for your coope...",0
2,2,ArdorNY.com-Manhattan's Best Value Apartment Rentals.\n=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=\nWEEKLY UPDATE OF OUR LATEST & NOTEWORTHY APARTMENTS.\nAs selected by our apartment rental specialists on 05/16/2001\n=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=...,0
3,3,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN""\n""http://www.w3.org/TR/html4/loose.dtd"">\n<html>\n<head>\n<title>Untitled Document</title>\n<meta http-equiv=""Content-Type"" content=""text/html; charset=iso-8859-1"">\n</head>\n\n<body>\n<p>&nbsp;</p>\n<p>Dear <strong>PayPal Inc. user,...",1
4,4,"solo chuks\n16 Kingsway Road \nIkoyi,Lagos.\n \nDear sir,\n\nGreetings to you and yourfamily. I will need your assistance in investing in \nyour country. Real Estate and importation into my country. But due to my \nposition as a civil servant I will need someone to assist me with this as my \njo...",1


#enron html data

In [85]:
# def extract_html_messages(df):
#     messages = []
#     for item in df["message"]:
#         # Return a message object structure from a string
#         e = email.message_from_string(item)
#         # get message body
#         message_body = e.get_payload()
#         if '<html>' in message_body:
#             messages.append(message_body)
#         # if BeautifulSoup(message_body, "html.parser").find():
#         #     messages.append(message_body)
#     print("Successfully retrieved message body from e-mails!")
#     print(len(messages))
#     return messages
# enron_html_bodies = extract_html_messages(emails)

In [86]:
# enron_html_bodies[0]


#positive: legit
#enron text:50W html:874
#iwspa text:5091 html:9

#negative: phishing
#fraudulent text:3978 html:147
#iwspa text:628 html:15
#jose text:10147 html:10147



#dataset:
#fradulent all 
#jose all html
#iwspa all no header
#enron 2W text + all html